# 3th_homework

### Import Section

In [1]:
import os

import numpy as np
import pandas as pd

from scipy.sparse import csr_matrix

# import implicit
from implicit.als import AlternatingLeastSquares

# import seaborn as sns
import plotly.graph_objects as go

### Global Settings Section

In [2]:
# Отключение многопоточности Numpy, чтобы избежать предупреждения от AlternatingLeastSquares:
# "WARNING:root:OpenBLAS detected."
# "Its highly recommend to set the environment variable 'export OPENBLAS_NUM_THREADS=1' to disable its internal multithreading"
os.environ['OPENBLAS_NUM_THREADS'] = '1'

# random_state_global = 0

### Path Section

In [3]:
PATH_DATA_RETAIL_TRAIN = r'../2th lesson/retail_train.csv'

## Theoretical Part

**1. Вспомним прошлый вебинар, мы рассматривали User-User рекомендации и Item-Item рекомендации. Чем они отличаются и чем они похожи? Если есть функция item_item_rec(interaction_matrix). Можно ли использовать эту функцию для user_user_rec?
В чем принципиальные отличия item-item рекомендаций от ALS?**

User-User рекомендации строятся на оценке эмбеддинга пользователя:
- эмбеддинг пользователя;
- поиск наиболее похожих пользователей;
- подготовка рекомендаций на основе усреднённой оценки пользователей.

Item-Item рекомендации, аналогично, используют эмбеддинг продукта:
- эмбеддинг продукта;
- поиск наиболее похожих продуктов;
- рекомендация новых товаров для пользователя.
    
Схожесть заключается в эмбеддинге и поиске похожих пользователей или продуктов. Различия - для продуктов достаточно рекомендовать наиболее похожие продукты согласно некой мере схожести, для пользователей нужна некая грруповая оценка новых продуктов.
    
Поэтому без доработок функцию вряд ли получится применить.
    
ALS отличается от Item-Item тем, что использует категоризацию пользователей и продуктов. Item-Item использует наиболее похожие продукты, а ALS наиболее близкие категории как для продуктов, так и для пользователей. Затем для пользователей с соответствующей категорией могут быть подобраны наиболее вероятные продукты из этой категории.

***

**2. Приведите 3 примера весов (те, которых не было на вебинаре: сумма покупок, количество покупок - неинтересно) user-item матрицы для задачи рекомендаций товаров**
    
1. Частота совершения действий клиентом. Например, количество обращений за услугой в месяц.
2. Непрерывность пользования сервисом. Например, число месяцев подряд когда клиент покупал абонемент или подписку.
3. Доля использования сервиса. Например, дочитал ли пользователь книгу или досмотрел фильм.

***

**3. Какие ограничения есть у ALS? (Тип информации, линейность/нелинейность факторов и т д)**
    
Так как метод ALS построен на факторизации матриц, основным требованием является возможность разложения матрицы. То есть, возможность найти сингулярные вектора матрицы, определитель матрицы и т.д. Работает только с положительными числами.
    
Достаточное количество и адекватность информации для сегментного представления пользователей и продуктов. При сильном дисбалансе сегментов работа метода будет затруднена.

## Practical Part

In [4]:
df_data = pd.read_csv(PATH_DATA_RETAIL_TRAIN)

In [5]:
df_data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [6]:
# Три последние недели будут использованы для валидации обученной модели.
test_size_weeks = 3

# Разбиение данных на обучающую и тестовую выборки.
df_data_train = df_data[df_data['week_no'] < df_data['week_no'].max() - test_size_weeks].copy()
df_data_test = df_data[df_data['week_no'] >= df_data['week_no'].max() - test_size_weeks].copy()

# Проверка вывод размерностей выборок.
df_data_train.shape, df_data_test.shape

((2278490, 12), (118314, 12))

In [7]:
# Формирование проверочной таблицы на обучающей выборке.
result_train = df_data_train.copy().groupby('user_id')['item_id'].unique().reset_index()
result_train.columns=['user_id', 'actual']
result_train.head(2)

,user_id,actual
0,1,"[825123, 831447, 840361, 845307, 852014, 85498..."
1,2,"[854852, 930118, 1077555, 1098066, 5567388, 55..."


In [8]:
# Формирование проверочной таблицы на валидачионной выборке.
result_test = df_data_test.copy().groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


#### Precision@K

In [9]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

#### Get Recommendations

In [10]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=[itemid_to_id[999999]], 
                                    recalculate_user=True)]
    return res

### Построение модели на основе признака "quantity"

#### Подготовка User-Item матрицы

In [11]:
# Формирование таблицы, где для каждого товара указано количество продаж.
popularity = df_data_train.copy().groupby('item_id')['quantity'].sum().reset_index()
popularity = popularity.rename(columns={'quantity': 'n_sold'})

# Составление списка 5000 наиболее продаваемых товаров.
top_5000 = popularity.copy().sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Товары вне топ 5000 определены в отдельную категорию.
df_data_train.loc[~df_data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [12]:
user_item_matrix = pd.pivot_table(df_data_train,
                                  index='user_id',
                                  columns='item_id', 
                                  values='quantity',
#                                   aggfunc='count', 
                                  aggfunc='sum',
                                  fill_value=0
                                 )

# Необходимый тип матрицы для implicit.
user_item_matrix = user_item_matrix.astype(float) 

# Перевод в формат saprse matrix.
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Словари перевода user_id и item_id к порядковым id и наоброт

In [13]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

#### Базовая модель из урока

In [14]:
%%time


model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result = result_test.copy()
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
metric_base_als = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
del result

metric_base_als

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 12.6 s


0.1694417238001959

Используем метрику базовой модели из урока для оценки перебора параметров.

#### Обучение модели при помощи перебора параметров

In [15]:
# Формирование таблицы гиперпараметров для подсчёта метрик
# столбцы - количество итераций,
# строки - коэффициенты ругеляризации.
# Данные параметры были получены методом перебора.
df_metrics = pd.DataFrame(np.zeros((5, 3)),
                          index=['0.1', '0.05', '0.01', '0.005', '0.001'],
                          columns=['10', '15', '20',])

# Проверка.
df_metrics

,10,15,20
0.1,0.0,0.0,0.0
0.05,0.0,0.0,0.0
0.01,0.0,0.0,0.0
0.005,0.0,0.0,0.0
0.001,0.0,0.0,0.0


In [16]:
%%time


# Формирования таблиц метрик для обучающей и валидационной выборок.
df_metrics_quantity_train = df_metrics.copy()
df_metrics_quantity_test = df_metrics.copy()

# Перебор гиперпараметров, указанных в таблице метрик.
for n_factors in df_metrics_quantity_train.columns:
    for reg_coeff in df_metrics_quantity_train.index:
        
        # Таблицы для оценки качества прогнозирования.
        result_train_temp = result_train.copy()
        result_test_temp = result_test.copy()
        
        # Инициализация модели с выбранными гиперпараметрами.
        model = AlternatingLeastSquares(factors=int(n_factors),
                                        regularization=float(reg_coeff),
                                        iterations=15,
                                        calculate_training_loss=True,
                                        use_gpu=False)
        
        # Обучение модели.
        model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
        
        # Прогнозирование о оценка качества на тернировочной выборке.
        result_train_temp['als'] = result_train_temp['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        df_metrics_quantity_train.loc[reg_coeff, n_factors] = (
            result_train_temp
            .apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1)
            .mean()        
        )
        del result_train_temp
        
        # Прогнозирование о оценка качества на валидационной выборке.
        result_test_temp['als'] = result_test_temp['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        df_metrics_quantity_test.loc[reg_coeff, n_factors] = (
            result_test_temp
            .apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1)
            .mean()        
        )   
        del result_test_temp

Wall time: 3min 45s


In [17]:
df_metrics_quantity_train

,10,15,20
0.1,0.626571,0.624810,0.614246
0.05,0.616887,0.604482,0.627051
0.01,0.601040,0.629772,0.639776
0.005,0.612965,0.633854,0.648579
0.001,0.608163,0.642337,0.654262


In [18]:
df_metrics_quantity_test

,10,15,20
0.1,0.177963,0.174633,0.168266
0.05,0.175710,0.166210,0.166112
0.01,0.172870,0.174339,0.170323
0.005,0.172772,0.173066,0.173849
0.001,0.168952,0.172478,0.174437


Анализ метрик качества обученной модели на основе признака "quantity" показал следующие особенности:
- качество прогнозирования на обучающей выборке гораздо лучше, чем на валидационной. Модель переобучилась;
- перебор гиперпараметров позволил получить метрики качества лучшие, чем метрика базовой модели.

### Построение модели на основе признака "sales_value"

#### Подготовка User-Item матрицы

In [19]:
# Три последние недели будут использованы для валидации обученной модели.
test_size_weeks = 3

# Разбиение данных на обучающую и тестовую выборки.
df_data_train = df_data[df_data['week_no'] < df_data['week_no'].max() - test_size_weeks].copy()
df_data_test = df_data[df_data['week_no'] >= df_data['week_no'].max() - test_size_weeks].copy()

# Проверка вывод размерностей выборок.
df_data_train.shape, df_data_test.shape

((2278490, 12), (118314, 12))

In [20]:
# Формирование таблицы, где для каждого товара указан объём продаж.
popularity = (
    df_data_train
    .copy()
    .groupby('item_id')['sales_value']
    .sum()
    .reset_index()
    .rename(columns={'sales_value': 'n_sold'})
)

# Составление списка 5000 наиболее продаваемых товаров.
top_5000 = popularity.copy().sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

# Товары вне топ 5000 определены в отдельную категорию.
df_data_train.loc[~df_data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [21]:
user_item_matrix = pd.pivot_table(df_data_train,
                                  index='user_id',
                                  columns='item_id', 
                                  values='sales_value',
#                                   aggfunc='count', 
                                  aggfunc='sum',
                                  fill_value=0
                                 )

# Необходимый тип матрицы для implicit.
user_item_matrix = user_item_matrix.astype(float) 

# Перевод в формат saprse matrix.
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,259120,397896,420647,480014,818981,819063,819255,819304,819308,819330,...,15926844,15926886,15971546,15972074,15972298,15972565,15972790,16053266,16100266,16769635
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,3.49,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Словари перевода user_id и item_id к порядковым id и наоброт

In [22]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

#### Базовая модель из урока

In [23]:
%%time


model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result = result_test.copy()
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
metric_base_als = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
del result

metric_base_als

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 12 s


0.11547502448579824

Используем метрику базовой модели из урока для оценки перебора параметров.

#### Обучение модели при помощи перебора параметров

In [24]:
# Формирование таблицы гиперпараметров для подсчёта метрик
# столбцы - количество итераций,
# строки - коэффициенты ругеляризации.
# Данные параметры были получены методом перебора.
df_metrics = pd.DataFrame(np.zeros((5, 3)),
                          index=['0.1', '0.05', '0.01', '0.005', '0.001'],
                          columns=['10', '15', '20',])

# Проверка.
df_metrics

,10,15,20
0.1,0.0,0.0,0.0
0.05,0.0,0.0,0.0
0.01,0.0,0.0,0.0
0.005,0.0,0.0,0.0
0.001,0.0,0.0,0.0


In [25]:
%%time


# Формирования таблиц метрик для обучающей и валидационной выборок.
df_metrics_sales_value_train = df_metrics.copy()
df_metrics_sales_value_test = df_metrics.copy()

# Перебор гиперпараметров, указанных в таблице метрик.
for n_factors in df_metrics_sales_value_train.columns:
    for reg_coeff in df_metrics_sales_value_train.index:
        
        # Таблицы для оценки качества прогнозирования.
        result_train_temp = result_train.copy()
        result_test_temp = result_test.copy()
        
        # Инициализация модели с выбранными гиперпараметрами.
        model = AlternatingLeastSquares(factors=int(n_factors),
                                        regularization=float(reg_coeff),
                                        iterations=15,
                                        calculate_training_loss=True,
                                        use_gpu=False)
        
        # Обучение модели.
        model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
        
        # Прогнозирование о оценка качества на тернировочной выборке.
        result_train_temp['als'] = result_train_temp['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        df_metrics_sales_value_train.loc[reg_coeff, n_factors] = (
            result_train_temp
            .apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1)
            .mean()        
        )
        del result_train_temp
        
        # Прогнозирование о оценка качества на валидационной выборке.
        result_test_temp['als'] = result_test_temp['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        df_metrics_sales_value_test.loc[reg_coeff, n_factors] = (
            result_test_temp
            .apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1)
            .mean()        
        )   
        del result_test_temp

Wall time: 3min 27s


In [26]:
df_metrics_sales_value_train

,10,15,20
0.1,0.540136,0.536935,0.535334
0.05,0.537495,0.532933,0.536935
0.01,0.542697,0.526451,0.536214
0.005,0.532693,0.525170,0.525010
0.001,0.537655,0.529572,0.530692


In [27]:
df_metrics_sales_value_test

,10,15,20
0.1,0.149265,0.141528,0.131146
0.05,0.150833,0.140744,0.130069
0.01,0.150049,0.135162,0.131342
0.005,0.148776,0.133105,0.128893
0.001,0.145250,0.134182,0.127816


Анализ метрик качества обученной модели на основе признака "sales_value" показал следующие особенности:
- качество прогнозирования на обучающей выборке гораздо лучше, чем на валидационной. Модель также переобучилась;
- перебор гиперпараметров позволил получить метрики качества лучшие, чем метрика базовой модели;
- качество базовой модели и моделей с перебором гиперпараметров хуже, чем у аналогичных моделей, обученных на основе признака "quantity".

### Построение модели на основе признака "retail_disc"

#### Подготовка User-Item матрицы

In [28]:
# Три последние недели будут использованы для валидации обученной модели.
test_size_weeks = 3

# Разбиение данных на обучающую и тестовую выборки.
df_data_train = df_data[df_data['week_no'] < df_data['week_no'].max() - test_size_weeks].copy()
df_data_test = df_data[df_data['week_no'] >= df_data['week_no'].max() - test_size_weeks].copy()

# Проверка вывод размерностей выборок.
df_data_train.shape, df_data_test.shape

((2278490, 12), (118314, 12))

In [29]:
# Указание модуля скидки для работы метода ALS.
df_data_train['retail_disc'] = np.abs(df_data_train['retail_disc'])
df_data_test['retail_disc'] = np.abs(df_data_test['retail_disc'])

In [30]:
# Формирование таблицы, где для каждого товара указана сумма скидок.
popularity = (
    df_data_train
    .copy()
    .groupby('item_id')['retail_disc']
    .sum()
#     .count()
    .reset_index()
)

# Составление списка 5000 наиболее продаваемых товаров.
top_5000 = popularity.copy().sort_values('retail_disc', ascending=False).head(5000).item_id.tolist()

# Товары вне топ 5000 определены в отдельную категорию.
df_data_train.loc[~df_data_train['item_id'].isin(top_5000), 'item_id'] = 999999

In [31]:
user_item_matrix = pd.pivot_table(df_data_train,
                                  index='user_id',
                                  columns='item_id', 
                                  values='retail_disc',
#                                   aggfunc='count', 
                                  aggfunc='sum',
                                  fill_value=0
                                 )

# Необходимый тип матрицы для implicit.
user_item_matrix = user_item_matrix.astype(float) 

# Перевод в формат saprse matrix.
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

item_id,27978,269894,323018,818981,819255,819308,819330,819400,819518,819549,...,16053242,16122432,16729299,16729363,16729415,16769972,16830135,17105257,17105530,17242362
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Словари перевода user_id и item_id к порядковым id и наоброт

In [32]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

#### Базовая модель из урока

In [33]:
%%time


model = AlternatingLeastSquares(factors=44, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                use_gpu=False)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

result = result_test.copy()
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
metric_base_als = result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
del result

metric_base_als

  0%|          | 0/15 [00:00<?, ?it/s]

Wall time: 10 s


0.13976493633692458

Используем метрику базовой модели из урока для оценки перебора параметров.

#### Обучение модели при помощи перебора параметров

In [34]:
# Формирование таблицы гиперпараметров для подсчёта метрик
# столбцы - количество итераций,
# строки - коэффициенты ругеляризации.
# Данные параметры были получены методом перебора.
df_metrics = pd.DataFrame(np.zeros((5, 3)),
                          index=['0.1', '0.05', '0.01', '0.005', '0.001'],
                          columns=['25', '30', '35',])

# Проверка.
df_metrics

,25,30,35
0.1,0.0,0.0,0.0
0.05,0.0,0.0,0.0
0.01,0.0,0.0,0.0
0.005,0.0,0.0,0.0
0.001,0.0,0.0,0.0


In [35]:
%%time


# Формирования таблиц метрик для обучающей и валидационной выборок.
df_metrics_retail_disc_train = df_metrics.copy()
df_metrics_retail_disc_test = df_metrics.copy()

# Перебор гиперпараметров, указанных в таблице метрик.
for n_factors in df_metrics_retail_disc_train.columns:
    for reg_coeff in df_metrics_retail_disc_train.index:
        
        # Таблицы для оценки качества прогнозирования.
        result_train_temp = result_train.copy()
        result_test_temp = result_test.copy()
        
        # Инициализация модели с выбранными гиперпараметрами.
        model = AlternatingLeastSquares(factors=int(n_factors),
                                        regularization=float(reg_coeff),
                                        iterations=15,
                                        calculate_training_loss=True,
                                        use_gpu=False)
        
        # Обучение модели.
        model.fit(csr_matrix(user_item_matrix).T.tocsr(), show_progress=False)
        
        # Прогнозирование о оценка качества на тернировочной выборке.
        result_train_temp['als'] = result_train_temp['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        df_metrics_retail_disc_train.loc[reg_coeff, n_factors] = (
            result_train_temp
            .apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1)
            .mean()        
        )
        del result_train_temp
        
        # Прогнозирование о оценка качества на валидационной выборке.
        result_test_temp['als'] = result_test_temp['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
        df_metrics_retail_disc_test.loc[reg_coeff, n_factors] = (
            result_test_temp
            .apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1)
            .mean()        
        )   
        del result_test_temp

Wall time: 3min 41s


In [36]:
df_metrics_retail_disc_train

,25,30,35
0.1,0.585354,0.612885,0.629932
0.05,0.583113,0.607043,0.633373
0.01,0.584954,0.611605,0.632573
0.005,0.590236,0.615926,0.632093
0.001,0.591597,0.615206,0.640336


In [37]:
df_metrics_retail_disc_test

,25,30,35
0.1,0.132125,0.135847,0.137512
0.05,0.132419,0.136435,0.136239
0.01,0.131342,0.132615,0.137218
0.005,0.130069,0.137512,0.140157
0.001,0.135162,0.135064,0.137610


Анализ метрик качества обученной модели на основе признака "retail_disc" показал следующие особенности:
- качество прогнозирования на обучающей выборке гораздо лучше, чем на валидационной. Модель также переобучилась;
- перебор гиперпараметров не дал существенного улучшения метрик зачества;
- аналогично "sales_value", качество базовой модели и моделей с перебором гиперпараметров хуже, чем у аналогичных моделей, обученных на основе признака "quantity".

### Выводы

Все модели продемонстрировали примерно одинаковые затраты времени на обучения при переборе гиперпараметров - около 3 с половиной минут.

Наилучшие метрики качества показала модель построенная на основе признака "quantity". Перебор гиперпараметров помог ещё больше улучшить метрику.

Для двух моделей "quantity" и "sales_value" перебор гиперпараметров показал улучшение метрик качества. Для модели "retail_disc" перебор не дал существенного улучшения результатов.